# LightlyTrain with RF-DETR

In this notebook we will demonstrate how you can use [LightlyTrain](https://docs.lightly.ai/train/stable/index.html) to pretrain an [RF-DETR model](https://github.com/roboflow/rf-detr). To this end, we will first use the raw images (**no labels**) from the [Coconuts dataset](https://universe.roboflow.com/traindataset/coconuts-plj8h) for pretraining and then fine-tuning on the labeled dataset.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/lightly-ai/lightly-train/blob/main/examples/notebooks/rfdetr.ipynb)

> **Important**: When running on Google Colab make sure to select a GPU runtime for faster processing. You can do this by going to `Runtime` > `Change runtime type` and selecting a GPU hardware accelerator.

## Installation

You can install `lightly_train` directly from PyPI using pip with support of `rfdetr` package.

In [ ]:
!pip install "lightly-train[rfdetr]"

> **Important**: LightlyTrain is officially supported on
> - Linux: CPU or CUDA
> - MacOS: CPU only
> - Windows (experimental): CPU or CUDA
>
> We are planning to support MPS for MacOS.
>
> Check the installation instructions for more details on installation.

## Download Dataset

For now, `rfdetr` only supports training with datasets in COCO JSON format. They can be directly downloaded via their API:


In [ ]:
from roboflow import Roboflow

rf = Roboflow(api_key="your_roboflow_api_key")

We use Roboflow's [Coconuts dataset](https://universe.roboflow.com/traindataset/coconuts-plj8h) for pretraining.

In [ ]:
project = rf.workspace("traindataset").project("coconuts-plj8h")
version = project.version(1)
pretrain_dataset = version.download("coco")

We use Roboflow's [Coconut Custom Dataset](https://universe.roboflow.com/ravi-mgvlz/coconut-custom-dataset) for fine-tuning.

In [ ]:
project = rf.workspace("ravi-mgvlz").project("coconut-custom-dataset")
version = project.version(3)
finetune_dataset = version.download("coco")

## Pretrain and Fine-tune an RF-DETR Model

Pretraining an RF-DETR model with LightlyTrain is straightforward:

### Pretrain

In [ ]:
import lightly_train

if __name__ == "__main__":
    lightly_train.train(
        out="out/my_experiment",                                        # Output directory.
        data=pretrain_dataset.location,                                 # Directory with images.
        model="rfdetr/rf-detr-base",                                    # Pass the RF-DETR model.
        epochs=5,                                                       # Number of epochs to train
        batch_size=16,                                                  # Batch size
        overwrite=True,
    )

### Fine-tune

You can use `rfdetr` for fine-tuning directly.

In [ ]:
from rfdetr import RFDETRBase

model = RFDETRBase(pretrain_weights="out/my_experiment/exported_models/exported_last.pt")
model.train(dataset_dir=finetune_dataset.location, epochs=10, batch_size=4, lr=1e-4)